In [ ]:
%pip uninstall openai-whisper
%pip install SpeechRecognition openai-whisper transformers pyaudio sentencepiece datasets[audio] ffmpeg-python pydub
# %pip install git+https://github.com/openai/whisper.git
# %apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

In [2]:
class Args:
    def __init__(self, model='medium', non_english=False, energy_threshold=1000, record_timeout=2, phrase_timeout=3, language='pt'):
        self.model = model
        self.non_english = non_english
        self.energy_threshold = energy_threshold
        self.record_timeout = record_timeout
        self.phrase_timeout = phrase_timeout
        self.language = language

args = Args(model='medium', non_english=True, energy_threshold=1000, record_timeout=2, phrase_timeout=3, language='pt')

In [3]:
import io
import os
import speech_recognition as sr
import whisper
import torch

from datetime import datetime, timedelta
from queue import Queue
import tempfile
from time import sleep
from sys import platform

In [5]:
def main():
    # The last time a recording was retrieved from the queue.
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    phrase_time = None
    # Current raw audio bytes.
    last_sample = bytes()
    # Thread safe Queue for passing data from the threaded recording callback.
    data_queue = Queue()
    # We use SpeechRecognizer to record our audio because it has a nice feature where it can detect when speech ends.
    recorder = sr.Recognizer()
    recorder.energy_threshold = args.energy_threshold
    # Definitely do this, dynamic energy compensation lowers the energy threshold dramatically to a point where the SpeechRecognizer never stops recording.
    recorder.dynamic_energy_threshold = False

    # Important for linux users.
    # Prevents permanent application hang and crash by using the wrong Microphone
    if 'linux' in platform:
        source = sr.Microphone(sample_rate=16000)
        mic_name = args.default_microphone
        if not mic_name or mic_name == 'list':
            print("Available microphone devices are: ")
            for index, name in enumerate(sr.Microphone.list_microphone_names()):
                print(f"Microphone with name \"{name}\" found")
            return
        else:
            for index, name in enumerate(sr.Microphone.list_microphone_names()):
                if mic_name in name:
                    source = sr.Microphone(sample_rate=16000, device_index=index)
                    break
    else:
        source = sr.Microphone(sample_rate=16000)

    # Load / Download model
    model = args.model
    if args.model != "large" and not args.non_english:
        model = model + ".en"
    audio_model = whisper.load_model(model)

    record_timeout = args.record_timeout
    phrase_timeout = args.phrase_timeout

    transcription = ['']

    with source:
        recorder.adjust_for_ambient_noise(source)

    def record_callback(_, audio:sr.AudioData) -> None:
        """
        Threaded callback function to receive audio data when recordings finish.
        audio: An AudioData containing the recorded bytes.
        """
        # Grab the raw bytes and push it into the thread safe queue.
        data = audio.get_raw_data()
        data_queue.put(data)

    # Create a background thread that will pass us raw audio bytes.
    # We could do this manually but SpeechRecognizer provides a nice helper.
    recorder.listen_in_background(source, record_callback, phrase_time_limit=record_timeout)

    # Cue the user that we're ready to go.
    print("Model loaded.\n")

    while True:
        try:
            now = datetime.utcnow()
            # Pull raw recorded audio from the queue.
            if not data_queue.empty():
                phrase_complete = False
                # If enough time has passed between recordings, consider the phrase complete.
                # Clear the current working audio buffer to start over with the new data.
                if phrase_time and now - phrase_time > timedelta(seconds=phrase_timeout):
                    last_sample = bytes()
                    phrase_complete = True
                # This is the last time we received new audio data from the queue.
                phrase_time = now

                # Concatenate our current audio data with the latest audio data.
                while not data_queue.empty():
                    data = data_queue.get()
                    last_sample += data

                # Use AudioData to convert the raw data to wav data.
                print('start take the audio data')
                audio_data = sr.AudioData(last_sample, source.SAMPLE_RATE, source.SAMPLE_WIDTH)
                wav_data = io.BytesIO(audio_data.get_wav_data())
                
                # temp_file = tempfile.NamedTemporaryFile(delete=False).name
                temp_file = tempfile.NamedTemporaryFile().name
                print(temp_file)
                with open(temp_file, 'w+b') as f:
                    f.write(wav_data.read())

                # Read the transcription.
                print("call model")
                result = audio_model.transcribe(temp_file, fp16=torch.cuda.is_available(), language=args.language)
                print(result['text'])
                text = result['text'].strip()

                # If we detected a pause between recordings, add a new item to our transcription.
                # Otherwise edit the existing one.
                if phrase_complete:
                    transcription.append(text)
                else:
                    transcription[-1] = text

                # Clear the console to reprint the updated transcription.
                for line in transcription:
                    print(line)
                # Flush stdout.
                print('', end='', flush=True)

                # Infinite loops are bad for processors, must sleep.
                sleep(0.25)
        except KeyboardInterrupt:
            break

    print("\n\nTranscription:")
    for line in transcription:
        print(line)


main()

 Mais com esse monte de print que eu também orantei desde a tradução, mas beleza, 0% de gente vai de novo.
Sei se eu...
Quase não passou tempo agora. Aí a pergunta que fica, né? Então, onde é que tá a TNC? Se não tiver base... E passou rapidinho.
Sei se eu... Tá... Tá indo bem ruim. Ei, ó... O que é que é?
Não, o que... Tendo em que agora ele tá assim... Uma floragem de coisa... Eu não sei porque... Não sei se é kill...
Mais com esse monte de print que eu também orantei desde a tradução, mas beleza, 0% de gente vai de novo.
start take the audio data
C:\Users\Diego\AppData\Local\Temp\tmp1z0ngoak
call model


Transcription:
Sei se eu...
Quase não passou tempo agora. Aí a pergunta que fica, né? Então, onde é que tá a TNC? Se não tiver base... E passou rapidinho.
Sei se eu... Tá... Tá indo bem ruim. Ei, ó... O que é que é?
Não, o que... Tendo em que agora ele tá assim... Uma floragem de coisa... Eu não sei porque... Não sei se é kill...
Mais com esse monte de print que eu também orantei de